In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.misc import imread
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.convolutional import Conv2D
from keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import sys

%matplotlib inline

Using TensorFlow backend.


In [2]:
bad_path = 'DataSets/Bad50x50/27733/'
left_path = 'DataSets/LeftHand50x50/27731/'
rigth_path = 'DataSets/RightHand50x50/27730/'
test_path = 'DataSets/Test189x110/27734/'

In [3]:
dir_path = [bad_path, left_path, rigth_path]

In [4]:
train_img_path = []
for i in dir_path :
    image_in_dir = []
    for j in os.listdir(i):
        if j.split('.')[1] == 'jpg':
            image_in_dir.append(i + j)
    train_img_path.append(image_in_dir)

In [5]:
target_list = [0, 1, 2]

In [6]:
train_data = []
train_target = []
for paths, target in zip(train_img_path, target_list):
    for path in paths:
        train_data.append(imread(path, mode='L'))
        train_target.append(target)
train_data = np.array(train_data)
train_target = np.array(train_target)

In [7]:
test_img_path = []
for i in os.listdir(test_path):
    if i.split('.')[1] == 'jpg':
        test_img_path.append(test_path + i)

In [8]:
test_data = []
for path in test_img_path:
    test_data.append(imread(path, mode='L', ))
test_data = np.array(test_data)

In [9]:
batch_size, img_rows, img_cols = 64 , 50, 50

In [10]:
input_shape = (img_rows, img_cols, 1)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(train_data, train_target)

In [12]:
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)

In [13]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [14]:
datagen_train = ImageDataGenerator(vertical_flip=True, rotation_range=45)
datagen_train.fit(X_train.reshape(-1, 50, 50, 1))
datagen_test = ImageDataGenerator(vertical_flip=True, rotation_range=45)
datagen_test.fit(X_test.reshape(-1, 50, 50, 1))

In [15]:
Y_train = np_utils.to_categorical(y_train, 3)
Y_test = np_utils.to_categorical(y_test, 3)

In [16]:
model = Sequential([
    Conv2D(32, (5, 5), input_shape=input_shape, padding='same'),
    Activation('relu'),
    MaxPooling2D(pool_size=(2,2), strides=(2, 2), padding ='same'),
    Conv2D(64, (5, 5), input_shape=input_shape, padding='same'),
    Activation('relu'),
    MaxPooling2D(pool_size=(2,2), strides=(2, 2), padding = 'same'),
    Flatten(),
    Dense(1024),
    Activation('relu'),
    Dropout(0.5),
    Dense(3),
    Activation('softmax')
])

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
model.fit_generator(datagen_train.flow(X_train, Y_train, batch_size=32),
                    steps_per_epoch=len(X_train) / 32,
                    validation_data=datagen_test.flow(X_test, Y_test, batch_size=32),
                    validation_steps = len(X_test) / 32,
                    epochs=10, 
                    verbose=1)

Epoch 1/10
53/52 [==============================] - 75s 1s/step - loss: 0.6246 - acc: 0.6859 - val_loss: 0.3363 - val_acc: 0.8656
Epoch 2/10
53/52 [==============================] - 82s 2s/step - loss: 0.3086 - acc: 0.8720 - val_loss: 0.1861 - val_acc: 0.9337
Epoch 3/10
53/52 [==============================] - 76s 1s/step - loss: 0.1902 - acc: 0.9333 - val_loss: 0.0726 - val_acc: 0.9803
Epoch 4/10
53/52 [==============================] - 68s 1s/step - loss: 0.0909 - acc: 0.9682 - val_loss: 0.0371 - val_acc: 0.9875
Epoch 5/10
53/52 [==============================] - 70s 1s/step - loss: 0.0931 - acc: 0.9667 - val_loss: 0.0796 - val_acc: 0.9749
Epoch 6/10
53/52 [==============================] - 73s 1s/step - loss: 0.0452 - acc: 0.9841 - val_loss: 0.0314 - val_acc: 0.9892
Epoch 7/10
53/52 [==============================] - 69s 1s/step - loss: 0.0325 - acc: 0.9870 - val_loss: 0.0294 - val_acc: 0.9910
Epoch 8/10
53/52 [==============================] - 68s 1s/step - loss: 0.0208 - acc: 0.99

In [19]:
mod_img = []
value = (31, 31)
l = 0
left = 0
right = 0
for img in test_data:
    blur = cv2.GaussianBlur(img, value, 0)
    _, thresh1 = cv2.threshold(blur, 127, 255,
                               cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    im2, contours, hierarchy = cv2.findContours(thresh1, 
                                                cv2.RETR_TREE,
                                                cv2.CHAIN_APPROX_SIMPLE)
    img_rect = img.copy()
    for i in contours:
        x, y, w, h = cv2.boundingRect(i)
        if (x, y) == (0, 0) or w < 20 or  h < 20:
            continue
        crop_img = img[y:y+h, x:x+w]
        crop_img_resize = cv2.resize(crop_img.copy(), (50,50), interpolation=cv2.INTER_AREA)
        pred = np.round(model.predict(crop_img_resize.reshape(-1,50, 50,1)))
        if np.all(pred == np.array([[0., 1., 0.]])):
            img_rect = cv2.rectangle(img_rect, (x,y), (x+w,y+h),255,1) 
            img_rect = cv2.putText(img_rect.copy(), 
                                        'left', 
                                        (x, y-1),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.4,255)
            left += 1
        if np.all(pred == np.array([[0., 0., 1.]])):
            img_rect = cv2.rectangle(img_rect, (x,y), (x+w,y+h),255,1)
            img_rect = cv2.putText(img_rect.copy(), 
                                        'right', 
                                        (x,y-1),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.4,255)
            right += 1
    mod_img.append(img_rect)
    if l % 50 == 0:
        print('IMG NUM: {}'.format(l))
    l += 1
print('\nTotal number of img: {}'.format(l))
print('Number of left hands: {}'.format(left))
print('Number of right hands: {}'.format(right))

IMG NUM: 0
IMG NUM: 50
IMG NUM: 100
IMG NUM: 150
IMG NUM: 200
IMG NUM: 250
IMG NUM: 300
IMG NUM: 350
IMG NUM: 400
IMG NUM: 450
IMG NUM: 500
IMG NUM: 550
IMG NUM: 600
IMG NUM: 650
IMG NUM: 700
IMG NUM: 750
IMG NUM: 800
IMG NUM: 850
IMG NUM: 900

Total number of img: 934
Number of left hands: 587
Number of right hands: 429


In [20]:
name_dir = 'updated_img'

In [21]:
os.mkdir(name_dir)

FileExistsError: [Errno 17] File exists: 'updated_img'

In [ ]:
os.chdir(name_dir)

In [ ]:
for i in range(len(mod_img)):
    plt.imsave(str(i)+'.png', mod_img[i], cmap='gray')